# Short-Time Fourier Transform

In [ ]:
import os
import numpy as np
from scipy.io import wavfile

def preprocess_audio_scipy(file_path, output_directory, target_sample_rate=22000, target_duration=2.0):
    """
    Preprocess audio using SciPy: convert to mono, downsample, trim/pad to 2 seconds, and normalize.
    Args:
        file_path (str): Path to the input audio file.
        output_directory (str): Directory to save the preprocessed file.
        target_sample_rate (int): Desired sampling rate for the audio.
        target_duration (float): Target duration in seconds.
    """
    # Load the audio file
    sample_rate, data = wavfile.read(file_path)

    # Convert to mono if stereo
    if len(data.shape) == 2:
        data = np.mean(data, axis=1).astype(data.dtype)

    # Downsample to the target sample rate
    if sample_rate != target_sample_rate:
        num_samples = int(len(data) * target_sample_rate / sample_rate)
        data = np.interp(
            np.linspace(0, len(data), num_samples, endpoint=False),
            np.arange(len(data)),
            data
        )
        sample_rate = target_sample_rate

    # Trim or pad the audio to the target duration
    target_length = int(sample_rate * target_duration)
    if len(data) > target_length:
        data = data[:target_length]
    elif len(data) < target_length:
        data = np.pad(data, (0, target_length - len(data)), mode='constant')

    # Normalize to zero mean and unit variance
    mean = np.mean(data)
    std = np.std(data)
    if std > 0:
        data = (data - mean) / std

    # Save the preprocessed file
    os.makedirs(output_directory, exist_ok=True)
    output_file = os.path.join(output_directory, os.path.basename(file_path))
    wavfile.write(output_file, sample_rate, data.astype(np.float32))
    print(f"Saved preprocessed file to: {output_file}")

def process_dataset_scipy(input_directory, output_directory):
    """
    Process all audio files in a directory using SciPy.
    Args:
        input_directory (str): Directory containing the original audio files.
        output_directory (str): Directory to save preprocessed files.
    """
    for file_name in os.listdir(input_directory):
        if file_name.endswith(".wav"):
            file_path = os.path.join(input_directory, file_name)
            preprocess_audio_scipy(file_path, output_directory)

# Define input and output directories
input_directory = "/Users/jakob/Downloads/IDMT_Traffic/audio"
output_directory = "/Users/jakob/Downloads/IDMT_Traffic/preprocessed_audio"

# Process the dataset
process_dataset_scipy(input_directory, output_directory)

/var/folders/vh/1t1ywg4n4fq7xwpclrv4jdyc0000gn/T/ipykernel_21567/4190772875.py:24: RuntimeWarning: invalid value encountered in divide
  spectral_centroid = np.sum(frequencies[:, None] * magnitude, axis=0) / np.sum(magnitude, axis=0)


Features saved to: /Users/jakob/Library/CloudStorage/OneDrive-student.kit.edu/Studium/02_Master/4. Semester/seminar/RTN-jakob/df_stft.xlsx


# Extract features to XLSX

In [10]:
import os
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy.signal import stft

def extract_features(file_path):
    """
    Extract audio features from a preprocessed WAV file.
    Args:
        file_path (str): Path to the audio file.
    Returns:
        dict: Extracted features.
    """
    # Load the preprocessed audio file
    sample_rate, data = wavfile.read(file_path)

    # Compute RMS amplitude
    rms = np.sqrt(np.mean(data**2))

    # Compute spectral centroid
    frequencies, times, Zxx = stft(data, fs=sample_rate, nperseg=512)
    magnitude = np.abs(Zxx)
    spectral_centroid = np.sum(frequencies[:, None] * magnitude, axis=0) / np.sum(magnitude, axis=0)
    mean_spectral_centroid = np.mean(spectral_centroid)

    # Compute energy in frequency bands
    low_energy = np.sum(magnitude[frequencies < 300])
    mid_energy = np.sum(magnitude[(frequencies >= 300) & (frequencies < 2000)])
    high_energy = np.sum(magnitude[frequencies >= 2000])

    # Strip .wav extension from file name
    file_name = os.path.basename(file_path).replace('.wav', '')

    # Return extracted features
    return {
        "file": file_name,  # Stripped of .wav
        "rms": rms,
        "spectral_centroid": mean_spectral_centroid,
        "low_energy": low_energy,
        "mid_energy": mid_energy,
        "high_energy": high_energy,
    }

def process_features(input_directory, output_file):
    """
    Process all audio files in a directory to extract features and save to Excel.
    Args:
        input_directory (str): Directory containing preprocessed audio files.
        output_file (str): Path to save the extracted features as an Excel file.
    """
    feature_list = []
    for file_name in os.listdir(input_directory):
        if file_name.endswith(".wav"):
            file_path = os.path.join(input_directory, file_name)
            features = extract_features(file_path)
            feature_list.append(features)
    
    # Convert to DataFrame and save to Excel
    df_features = pd.DataFrame(feature_list)
    df_features.to_excel(output_file, index=False)
    print(f"Features saved to: {output_file}")

# Define input and output paths
input_directory = "/Users/jakob/Downloads/IDMT_Traffic/preprocessed_audio"
output_file = "/Users/jakob/Library/CloudStorage/OneDrive-student.kit.edu/Studium/02_Master/4. Semester/seminar/RTN-jakob/df_stft.xlsx"

# Process and save features
process_features(input_directory, output_file)

/var/folders/vh/1t1ywg4n4fq7xwpclrv4jdyc0000gn/T/ipykernel_21567/3167945768.py:24: RuntimeWarning: invalid value encountered in divide
  spectral_centroid = np.sum(frequencies[:, None] * magnitude, axis=0) / np.sum(magnitude, axis=0)


Features saved to: /Users/jakob/Library/CloudStorage/OneDrive-student.kit.edu/Studium/02_Master/4. Semester/seminar/RTN-jakob/df_stft.xlsx


# Merge df_stft.xlsx with df_dataset.xlsx

In [1]:
import pandas as pd

# Load datasets
df_dataset = pd.read_excel("df_dataset.xlsx")
df_stft = pd.read_excel("df_stft.xlsx")

# Merge datasets on 'file' column (left join to retain all rows in df_dataset)
df_merged = pd.merge(df_dataset, df_stft, on="file", how="left")

# Save the merged dataset
df_merged.to_excel("df_dataset_merged.xlsx", index=False)
print("Merged dataset saved as df_dataset_merged.xlsx")

Merged dataset saved as df_dataset_merged.xlsx
